In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [2]:
URL = "https://www.mandy.com/uk/jobs/music/page/1"
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
page = requests.get(URL, headers=agent)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-GB" xml:lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="en-GB" name="language"/>
  <meta content="info@mandy.com" name="author"/>
  <meta content="The Mandy Network Ltd 2017 - 2020" name="copyright"/>
  <meta content="7944563C137DAEF626BF314084C7D089" name="msvalidate.01"/>
  <meta content="e03a16f38bb512de96fdb6f0f38a9e9e" name="p:domain_verify"/>
  <meta content="100000977468631,100002126792246" property="fb:admins"/>
  <meta content="GB" name="geo.region"/>
  <meta content="en_GB" property="og:locale"/>
  <meta content="Mandy" property="og:site_name"/>
  <meta content="website" property="og:type"/>
  <meta content="https://www.mandy.com/assets/images/mandy/logo-square-white-bg.jpg" property="og:image"/>
  <meta content="image/png" property="og:image:type"/>
  <meta content="Latest Music Jobs in UK, Mandy Musi

In [3]:
def extract_job_titles_from_result(soup): 
    job_titles = []
    spans = soup.findAll("ul", attrs={"class": "search-results","id":"job-results"})
    for span in spans:
        h2s = span.findAll("h2")
        for h2 in h2s:
            job_titles.append(h2.text.strip())
    return(job_titles)
job_titles = extract_job_titles_from_result(soup)

In [4]:
print(job_titles)
print(len(job_titles))

['Visiting Music Teacher - Piano', 'Junior Intercompany Royalty Analyst', 'Producer', 'Development Manager (Individuals)', 'Lecturer in Music', 'Teacher of Music', 'Music Administrator and Concert Manager', 'Visiting Music Teacher - Flute', 'Lead Practitioners – Voice / Technical Sound', 'Head of Marketing', 'Administrator & Projects Co-Ordinator', 'Learning and Development Business Partner', 'Piano Tutor', 'Music Teacher', 'Head of Label Partnerships', 'Teacher of Music', 'Vocalists and Percussionists', 'Release Manager', 'Teacher of Music and Performing Arts', 'Teacher of Music']
20


In [5]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text.strip())
    return(locations)
locations = extract_location_from_result(soup)

In [6]:
print(locations)
print(len(locations))

['Horsham', 'London', 'London', 'London', 'Plymouth', 'Frinton-on-Sea', 'Guildford', 'Horsham', 'London', 'Chichester', 'Preston', 'London', 'London', 'London', 'London', 'Nuneaton', 'Manchester', 'London', 'Rishworth', 'Aylesford']
20


In [7]:
def extract_jobType_from_result(soup): 
    job_types = []
    spans = soup.findAll("span", attrs={"class": "type"})
    for span in spans:
        job_types.append(span.text.strip())
    return(job_types)
job_types = extract_jobType_from_result(soup)

In [8]:
print(job_types)
print(len(job_types))

['Education', 'Administrative', 'Production / Engineering', 'Administrative', 'Teacher', 'Teacher', 'Administrative', 'Education', 'Teacher', 'Administrative', 'Administrative', 'Administrative', 'Teacher', 'Teacher', 'Other', 'Teacher', 'Theatre', 'Administrative', 'Education', 'Teacher']
20


In [15]:
mandy = pd.DataFrame()

def extractor(range1, range2):
    columns=['jobTitle','location', 'jobType']

    final = pd.DataFrame()

    for page_number in range(range1,range2):
        mandy = pd.DataFrame(columns=columns)
        
        agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
        page = requests.get("https://www.mandy.com/uk/jobs/music/page/" + str(page_number), headers=agent)
        time.sleep(5)
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")

        job_titles = extract_job_titles_from_result(soup)
        locations = extract_location_from_result(soup)
        job_types = extract_jobType_from_result(soup)
        
        mandy['jobTitle'] = list(job_titles)
        mandy['location'] = list(locations)
        mandy['jobType'] = list(job_types)

        final = final.append(mandy)
    
    return final
#https://stackoverflow.com/questions/44865673/access-denied-while-scraping

In [16]:
mandy = extractor(1, 5)

In [17]:
print(mandy.shape)

(80, 3)


In [18]:
mandy.head()

jobTitle  location                   jobType
0       Visiting Music Teacher - Piano   Horsham                 Education
1  Junior Intercompany Royalty Analyst    London            Administrative
2                             Producer    London  Production / Engineering
3    Development Manager (Individuals)    London            Administrative
4                    Lecturer in Music  Plymouth                   Teacher

In [19]:
mandy['jobTitle'].value_counts()

Teacher of Music                                          5
Piano Tutor                                               5
Development Manager (Individuals)                         2
Sound Designer                                            2
Music Teacher                                             2
Violin Teacher                                            2
Technical Stage Staff                                     1
Producer                                                  1
Administrative Manager                                    1
HR Admin Support                                          1
Director of Photography                                   1
Visiting Music Teacher – (Piano)                          1
Theatre Administrator                                     1
Session Musicians                                         1
Visiting Music Teacher - Piano                            1
Head of Programmes                                        1
Head of Label Partnerships              

In [20]:
mandy['location'].value_counts()

London                                  31
Sheffield                                4
Manchester                               4
York                                     3
Between Cambridge and Norwich, UK        2
Edinburgh                                2
Horsham                                  2
Chichester                               2
Rishworth                                1
Aylesford                                1
Salford, Greater Manchester              1
W1                                       1
Upminster                                1
Plymouth                                 1
South Petherton                          1
Crawley                                  1
Chelmsford                               1
Preston                                  1
Amersham                                 1
Tramshed                                 1
Dartford                                 1
world wide                               1
Milton Keynes                            1
Glasgow    

In [21]:
mandy['jobType'].value_counts()

Administrative              33
Teacher                     24
Education                    5
Film (Short)                 4
Digital                      3
Theatre                      2
Live Performance             2
Cruise                       1
Commercial                   1
Online / Digital             1
Musician                     1
Other                        1
Online Music Service         1
Production / Engineering     1
Name: jobType, dtype: int64

In [22]:
mandy.to_csv("mandy.csv", encoding="utf-8")


Source for all Scraping:

https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b